In [15]:
# Import the necessary libraries

from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from IPython.display import Markdown, display

In [2]:
# Initialize the environment

load_dotenv(override=True)

True

In [ ]:
# Let's just check emails are working for us

# def send_test_email():
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("gpallapa@pm.me")  # Change to sendgrid's verified sender
#     to_email = To("gpallapa@pm.me")  # Change to the recipient
#     content = Content("text/plain", "This is an important test email")
#     mail = Mail(from_email, to_email, "Test email", content).get()
#     response = sg.client.mail.send.post(request_body=mail)
#     print(response.status_code)

# send_test_email()

# If we receive a 202, we know the email was sent successfully

202


In [6]:
# Lets create the instructions for the sales agent

instructions1 = "You are a sales agent working for BeansAI, \
a company that provides global investment banking and wealth management services, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for BeansAI, \
a company that provides global investment banking and wealth management services, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for BeansAI, \
a company that provides global investment banking and wealth management services, powered by AI. \
You write concise, to the point cold emails."

In [7]:
# Lets create the agents each with different personas

sales_agent1 = Agent(
    name="Professional Sales Agent 1",
    instructions=instructions1,
    model="gpt-4o-mini"
)

sales_agent2 = Agent(
    name="Engaging Sales Agent 2",
    instructions=instructions2,
    model="gpt-4o-mini"
)

sales_agent3 = Agent(
    name="Busy Sales Agent 3",
    instructions=instructions3,
    model="gpt-4o-mini"
)

In [ ]:
# Approach if we want to stream the response

# result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
# async for event in result.stream_events():
#     if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
#         print(event.data.delta, end="", flush=True)

In [10]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message)
    )

    outputs = [result.final_output for result in results]

    for output in outputs:
        print(output + "\n\n")
    


Subject: Unlock Your Investment Potential with AI-Powered Insights

Dear [Recipient's Name],

I hope this message finds you well.

I am reaching out to introduce you to BeansAI, where we leverage advanced artificial intelligence to redefine the landscape of investment banking and wealth management. Our innovative solutions are designed to enhance decision-making processes, optimize portfolio management, and ultimately drive superior outcomes for our clients.

In an era where data-driven insights are paramount, our AI technology empowers investors like you to stay ahead of market trends and capitalize on emerging opportunities. Whether your focus is on maximizing returns, managing risk, or exploring new investment avenues, BeansAI provides tailored strategies to align with your specific goals.

Here are a few ways we can support your investment journey:

1. **Predictive Analytics**: Our AI algorithms analyze vast amounts of data to forecast market movements, giving you a strategic edge.

In [11]:
# Create a sales picker agent which picks the best sales agent based on the input

sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [17]:
# Now use the sales picker agent to pick the best sales agent based on the input

message = "Write a cold sales email touting the benefits of our AI-powered investment banking and wealth management services. \
    Be formal and professional at all times. You are emailing millionaires and High Net Worth Individuals who are interested in investing in our services, \
        and so you will be a brand ambassador for the company. \
            For our company's website, you can use the following URL: https://www.beansai.com in your email outreach."

with trace("Call the various sales agents in parallel and pick the best one"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message)
    )

    outputs = [result.final_output for result in results]

    # Consolidate all the emails into a single email
    consolidated_emails = "Cold sales emails:\n\n".join(outputs)

    # Pick the best one using the sales picker agent
    best_email = await Runner.run(sales_picker, consolidated_emails)

    print(f"Best sales email to use: \n\n{best_email.final_output}")
    

Best sales email to use: 

Subject: Elevate Your Wealth with Intelligent Insights

Dear [Recipient's Name],

I hope this email finds you in great spirits! I wanted to take a moment to introduce you to something that could elevate your wealth management experience to new heights.

At BeansAI, we understand that managing wealth is not just about numbers—it's about crafting your legacy. Our AI-powered investment banking and wealth management services are designed to transform your investment journey, providing intelligent insights that make a tangible difference to your portfolio.

Imagine having a dedicated team of cutting-edge algorithms analyzing market trends, identifying investment opportunities, and optimizing your wealth strategy while you enjoy a well-deserved round of golf (or whatever your leisure pursuit may be!). Our technology enables us to tailor solutions specifically to your needs, ensuring that your financial aspirations are not just met, but exceeded.

Here are a few ben

In [18]:
display(Markdown(best_email.final_output))

Subject: Elevate Your Wealth with Intelligent Insights

Dear [Recipient's Name],

I hope this email finds you in great spirits! I wanted to take a moment to introduce you to something that could elevate your wealth management experience to new heights.

At BeansAI, we understand that managing wealth is not just about numbers—it's about crafting your legacy. Our AI-powered investment banking and wealth management services are designed to transform your investment journey, providing intelligent insights that make a tangible difference to your portfolio.

Imagine having a dedicated team of cutting-edge algorithms analyzing market trends, identifying investment opportunities, and optimizing your wealth strategy while you enjoy a well-deserved round of golf (or whatever your leisure pursuit may be!). Our technology enables us to tailor solutions specifically to your needs, ensuring that your financial aspirations are not just met, but exceeded.

Here are a few benefits that make our services truly remarkable:

1. **Data-Driven Decisions**: Our AI analyzes vast amounts of data, delivering insights that human intuition might miss. It’s like having a crystal ball, but with a much better internet connection!
   
2. **Personalized Investment Strategies**: We recognize that no two millionaires are alike. Our tailored solutions cater specifically to your financial goals and risk tolerance.

3. **Real-time Insights**: Our platform provides you with real-time updates, so you're never out of the loop—unless, of course, you’re on a private yacht!

4. **Sustainable Growth**: We prioritize investments that not only serve your financial goals but also contribute positively to the world.

If you're ready to revolutionize your investment journey and explore the powerful potential of AI in wealth management, I would love to schedule a time for a conversation. You can also learn more about our innovative approach on our website: [BeansAI](https://www.beansai.com).

Thank you for considering BeansAI as your partner in achieving exceptional financial success. I look forward to the possibility of working together to grow your wealth.

Warm regards,

[Your Name]  
[Your Title]  
BeansAI  
[Your Phone Number]  
[Your Email]  

In [ ]:
# Now send the best email to the user

# Create a SendGrid client
# sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))

# # Create a Mail object
# mail = Mail(
#     from_email=Email("gpallapa@pm.me"),
#     to_emails=To("gpallapa@pm.me"),
#     subject="Test Email",
#     html_content=best_email.final_output
# )

In [68]:
# Create a function tool to send an email

# @function_tool
# def send_email(body: str):
#     """
#     Send an email with the given body to all sales prospects
#     """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))

#     from_email = Email("gpallapa@pm.me")
#     to_emails = To("gpallapa@pm.me")
    

#     # Now strip out the subject from the response
#     lines = body.split('\n', 1)
#     subject = lines[0].replace('Subject: ', '') if lines[0].startswith('Subject:') else 'BeansAI Sales Email'
#     body = lines[1].strip() if len(lines) > 1 else body
#     content = Content("text/plain", body)

#     print(f"Subject: {subject}\n\nBody: {content}")

#     mail = Mail(from_email, to_emails, subject, content).get()

#     response = sg.client.mail.send.post(request_body=mail)

#     if response.status_code == 202:
#         return "Email sent successfully"
#     else:
#         return "Email failed to send"


@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gpallapa@pm.me")  
    to_email = To("gautham.pallapa@scotiabank.com")  

    # Now strip out the subject from the response
    lines = body.split('\n', 1)
    subject = lines[0].replace('Subject: ', '') if lines[0].startswith('Subject:') else 'BeansAI Sales Email'
    body = lines[1].strip() if len(lines) > 1 else body
    content = Content("text/plain", body)

    # print(f"Subject: {subject}\n\nBody: {content}")

    mail = Mail(from_email, to_email, subject, content).get()

    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [60]:
# Create the sales agents as tools


tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="A sales agent that writes professional, serious cold emails")
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="A sales agent that writes witty, engaging cold emails")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="A sales agent that writes concise, to the point cold emails")

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='A sales agent that writes professional, serious cold emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110ba9440>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='A sales agent that writes witty, engaging cold emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110baa020>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='A sales agent that writes concise, to the point cold ema

In [65]:
# Now create a Sales Manager agent - the planning agent that uses the sales agents as tools and picks the best one

instructions ="You are a sales manager working for BeansAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)
    print(result.final_output)

The cold sales email has been successfully sent to "Dear CEO." If you need anything else, feel free to ask!


In [79]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [91]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gpallapa@pm.me")  
    to_email = To("gautham.pallapa@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    
    return {"status": "success"}

In [81]:
tools = [subject_tool, html_tool, send_html_email]

In [83]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [84]:
tools = [tool1, tool2, tool3, send_email]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='A sales agent that writes professional, serious cold emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110ba9440>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='A sales agent that writes witty, engaging cold emails', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x110baa020>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='A sales agent that writes concise, to the point cold email

In [92]:
sales_manager_instructions = "You are a sales manager working for BeansAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice. Ensure that the email body starts with Dear CEO and is not a mail merge filler field."

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)